<a href="https://colab.research.google.com/github/gborn/Whatsapp_Chat_Analysis/blob/main/notebooks/03-webapp-and-deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of WhatsApp Conversations - WebApp and Deployment

In this final notebook, we create a streamlit webapp, and deploy it to heroku. All the steps are documented in the notebook.
<br>
The webapp has been deployed at [chatresults.herokuapp.com](https://chatresults.herokuapp.com/)




## Streamlit WebApp

<br>
We create three files preprocessing.py, utils.py, and app.py.<br><br>
preprocessing.py reads an exported WhatsApp chats text file, and performs preprocessing steps outlined in data wrangling notebook.<br><br>
utils.py has helpful helper functions to filter data and plot visualizations <br> <br>
app.py renders a Streamlit web app UI and is the main entry point for our application

In [1]:
# install all necessary libraries
!pip install streamlit > /dev/null
!npm install -g localtunnel > /dev/null
!pip install fire > /dev/null
!pip install urlextract > /dev/null
!pip install wordcloud > /dev/null
!pip install emoji > /dev/null
!pip install altair > /dev/null
!pip install numerize > /dev/null
!pip install top2vec > /dev/null

In [11]:
!mkdir -p utils

In [5]:
%%writefile utils/preprocessing.py

import pandas as pd
import re
import csv
import fire
import warnings
warnings.filterwarnings("ignore")

def separator(msg):
    """
    Extract datetime, person name or number, and message text from msg
    """
    # remove whitespaces
    msg = msg.strip()

    # we define three groups: datetime, person name/phone number, message
    result = re.search(r'(\d{1,2}/\d{1,2}/\d{4},\s+\d{1,2}:\d{1,2})\s+-\s+([+0-9a-zA-Z\s]+):\s+(.*)', msg)

    # ignore lines that don't have above three groups, and return empty line
    if not hasattr(result, 'group'):
        return ''

    return result.groups()




def txt_to_csv(fn):
    """
    Read lines from text file "fn",
    create and save a csv file
    """
    with open(fn) as f:
         lines = map(separator, f.readlines())

    with open(f'{fn[:-4]}.csv', 'w') as wf:
        out = csv.writer(wf)
        out.writerow(['datetime', 'id', 'message'])
        for line in lines:
            if line:
                out.writerow(line)





def add_datepart(df, fieldname):
    """
    Adds date related features to dataframe df inplace
    df: dataframe
    fieldname: name of the date field in df
    """
    new_df = df.copy()
    field = df[fieldname]
    target_prefix = re.sub('[Dd]atetime$', '', fieldname)
    
    date_features = (
         'hour',
         'minute',
         'Year', 
         'Month', 
         'Week', 
         'Day', 
         'Dayofweek', 
         'Dayofyear', 
    )
    
    for name in date_features:
        new_df[target_prefix+name] = getattr(field.dt, name.lower())
        
    new_df[target_prefix+'Elapsed'] = (field - field.min()).dt.days
    new_df[target_prefix+'MonthName'] = field.dt.month_name()
    new_df[target_prefix+'DayName'] = field.dt.day_name()
    new_df.drop(fieldname, axis=1, inplace=True)

    return new_df




def preprocess(fn:str) -> pd.DataFrame:
    """
    Preprocess whatsapp text file
    """
    txt_to_csv(fn)
    chats_df = pd.read_csv(f'{fn[:-4]}.csv', parse_dates=['datetime'])
    chats = chats_df.set_index('datetime')

    chats_with_features = add_datepart(chats_df, 'datetime')

    return chats, chats_with_features

if __name__ == '__main__':
    import sys
    if sys.argv:
        chats_df = preprocess(sys.argv[1])
        print(chats_df.head(5))

Overwriting utils/preprocessing.py


In [28]:
%%writefile utils/helpers.py
import pandas as pd
from collections import Counter
from wordcloud import WordCloud
import emoji
import altair as alt
from urlextract import URLExtract
import plotly.graph_objects as go

urlextractor = URLExtract()

def fetch_messages(df, user):
    """
    Returns messages of selected user
    """
    if user.lower() != 'overall':
        df = df[df.id == user]

    df = df[df.message != '<Media omitted>']
    df = df[df.message != 'This message was deleted']

    return df




def fetch_stats(df, user):
    """
    Returns stats on number of messages, members, media files, links shared
    """
    if user.lower() != 'overall':
        df = df[df.id == user]

    # 1. fetch number of messages
    num_messages = df.message.shape[0]

    # 2. count number of words, and
    # 3. number of urls
    words = []
    urls = []
    for message in df.message:
        words.extend(message)
        urls.extend(urlextractor.find_urls(message))

    num_words = len(words)
    num_links = len(urls)

    # 4. number of media files shared
    num_medias = df[df.message == '<Media omitted>'].shape[0]
    
    return num_messages, num_words, num_medias, num_links




def fetch_active_users(df):
    """
    Return dataframe on most active users
    """
    new_df = (df['id'].value_counts()
                    .reset_index()
                    .rename(columns={'index': 'User', 'id':'Messages'})
                    .sort_values(by='Messages', ascending=False)
    )

    active_users_percent = ( (df.id.value_counts()/int(df.shape[0]) * 100)
                                .apply(lambda x: f'{x:.2f}')
                                .reset_index()
                                .rename(columns={'index': 'User', 'id':'Messages(%)'})
                                .sort_values(by='Messages(%)', ascending=False)
                            )

    return new_df, active_users_percent




def get_wordcloud(df, user):
    """
    Generates word cloud
    """
    if user.lower() != 'overall':
        df = df[df.id == user]

    df = df[df.message != '<Media omitted>']
    df = df[df.message != 'This message was deleted']

    wc = WordCloud(width=700, height=300, min_font_size=12, background_color='white')
    wc = wc.generate(df['message'].str.cat(sep=' '))
    return wc





def most_common_emojis(df, user, n=10):
    """
    Tokenize each message, and build list of nouns, verbs, phrases 
    to return "n" most common words, and emojis
    """
    if user.lower() != 'overall':
        df = df[df.id == user]

    df = df[df.message != '<Media omitted>']
    df = df[df.message != 'This message was deleted']
    
    # tokenize
    tokens = [word for msg in df.message for word in msg.split()]

    # filter emojis
    emojis = [word for word in tokens if word in emoji.UNICODE_EMOJI['en']]
    common_emojis = Counter(emojis).most_common(n)
    del emojis
 
    # create a dataframe and build a barchart
    def to_barchart(table):
        df = pd.DataFrame(table)
        df = df.rename(columns={0: 'Phrases', 1:'Count'})
        chart = _get_barchart(df, 'Count','Phrases', 'Phrases', 'Count')
        del df
        return chart

    return  to_barchart(common_emojis)




def _get_barchart(df, x, y, color, label):
    """
    helper function to build a barchart
    """
    bar_chart = alt.Chart(df).mark_bar(
                        cornerRadiusTopLeft=3,
                        cornerRadiusTopRight=3,
                ).encode(
                        x=alt.X(x, axis=alt.Axis(title=None)),
                        y=alt.Y(y, axis=alt.Axis(title=None)),
                        color=alt.Color(color, legend=None),
                )

    text = bar_chart.mark_text(
                align='center',
                dx=9,
                color='white'
                ).encode(
                text=label
            )
    return bar_chart + text




def timeline_stats(df, user):
    """
    Return timespan of messages, first message date, and last message date
    """
    if user.lower() != 'overall':
        df = df[df.id == user]

    df = df[df.message != '<Media omitted>']
    df = df[df.message != 'This message was deleted']

    if user.lower() == 'overall':
        total_days = df.Elapsed.max() 
    else:
        total_days = df.groupby(df.Elapsed)['message'].count().shape[0]

    first_date = df.iloc[0, [7, 5, 4]].to_dict()
    first_date = f"{first_date['Day']}-{first_date['Month']}-{first_date['Year']}"

    last_date = df.iloc[-1, [7, 5, 4]].to_dict()
    last_date = f"{last_date['Day']}-{last_date['Month']}-{last_date['Year']}"

    return total_days, first_date, last_date




def get_timelines(df, user):
    """
    Build line chart to showcase yearly timeline, 
    and chart charts to show  most active months, day of week, and hour of day
    """
    if user.lower() != 'overall':
        df = df[df.id == user]

    df = df[df.message != '<Media omitted>']
    df = df[df.message != 'This message was deleted']

    # timelines
    yearly_timeline = df.resample('M')['message'].count().reset_index()
    yearly_text = df.resample('M')['message'].apply(pd.Series.mode).tolist()

    df_emojis = df['message'].apply(lambda lst:[x if x in emoji.UNICODE_EMOJI['en'] else -1 for x in lst][0])
    df_emojis = df[df_emojis!=-1]

    levels = ['datetime']
    for idx in range(df_emojis.index.nlevels-1):
        levels.append(idx)

    df_emojis = df_emojis['message'].resample('M').apply(pd.Series.mode).reset_index(level=levels)

    daily_timeline = df.resample('D')['message'].count().reset_index()
    daily_timeline = daily_timeline[daily_timeline['message'] != 0]
    daily_text = df.resample('D')['message'].apply(pd.Series.mode).tolist()

    # most active days, and hours
    hourly_timeline = df.groupby([df.index.hour])['message'].count().reset_index()
    hourly_text = df.groupby([df.index.hour])['message'].apply(pd.Series.mode).reset_index()['message'].tolist()

    weekly_timeline = df.groupby([df.index.day_name()])['message'].count().reset_index()
    weekly_text = df.groupby([df.index.day_name()])['message'].apply(pd.Series.mode).reset_index()['message'].tolist()

    # yearly timeline displaying total messages in each month-year
    monthly_fig = go.Figure()
    monthly_fig.add_trace(go.Scatter(
        x=yearly_timeline['datetime'], 
        y=yearly_timeline['message'],
        hovertemplate =
        '<b>%{y:.2s} messages</b>: '+
        '<br><i>%{text}</i>',
        text = yearly_text,
        name='Timeline of emoticons'
    ))

    monthly_fig.add_trace(go.Bar(
        x=yearly_timeline['datetime'], 
        y=yearly_timeline['message'],
        hovertemplate="%{y:.2s}",
        name='Number of Messages',
        marker=dict(color=yearly_timeline['message'], colorbar=None),
    ))

    emoji_title = 'Timeline of messages (month-wise)' 
    if len(df_emojis) >= len(yearly_timeline):
        emoji_title = 'Evolution of Emoticons over time'
        

    monthly_fig.update_layout(
                        title=emoji_title,
                        yaxis= go.layout.YAxis(title="Total Messages"),
                        showlegend=False,
                        xaxis = go.layout.XAxis(title='Months', tickangle=45),
                        xaxis_tickformat = '%B<br>%Y',
                        autosize=False,
                        height=500,
                   )
    
    monthly_fig.update_xaxes(
        rangeslider_visible=True,
        )
    

    monthly_fig.add_trace(go.Scatter(
        x=df_emojis['datetime'], 
        y=yearly_timeline.set_index('datetime').loc[df_emojis.datetime, 'message'],
        text=df_emojis['message'].tolist(),
        mode="markers+text",
        name='',
    ))

    # daily timeline of messages, displays hover text
    daily_fig = go.Figure([
                           go.Scatter(
                               x=daily_timeline['datetime'], 
                               y=daily_timeline['message'],
                               hovertemplate = 
                               '<b>%{y:.2s} messages</b>:' +
                               '<br><i>%{text}</i>',
                                text = daily_text,
                                name='',
    )])

    daily_fig.update_layout(
                        title='Timeline of messages (day-wise)',
                        yaxis= go.layout.YAxis(title="Total Messages"),
                        showlegend=False,
                        xaxis = go.layout.XAxis(title='Days', tickangle=45),
                        xaxis_tickformat = '%d %B (%a)<br>%Y',
                        autosize=False,
                        height=500,
                   )
    
    daily_fig.update_xaxes(
        rangeslider_visible=True,
        )
    
    
    # most active days
    weekly_fig = go.Figure([go.Bar(
                                x=weekly_timeline['datetime'], 
                                y=weekly_timeline['message'],
                                hovertemplate =
                                   '<b>%{y:.2s} messages</b>: '+
                                   '<br><i>%{text}</i>',
                                text = weekly_text,
                                name='',
                                marker=dict(color=weekly_timeline['message'], colorbar=None),
                                
                )])
    weekly_fig.update_layout(
                        title='Most Active Days',
                        yaxis= go.layout.YAxis(title="Total Messages"),
                        showlegend=False,
                        xaxis = go.layout.XAxis(title='Day of the Week', tickangle=45)
                   )
    
    weekly_fig.update_traces(texttemplate='%{y:.2s}', textposition='outside')

    
    # most active hours
    hourly_fig = go.Figure([go.Bar(
                                x=hourly_timeline['datetime'], 
                                y=hourly_timeline['message'],
                                hovertemplate = 
                                    '<b>%{y:.2s} messages</b>: '+
                                    '<br><i>%{text}</i>',
                                text = hourly_text,
                                name='',
                                marker=dict(color=hourly_timeline['message'], colorbar=None),
                                
                )])
    hourly_fig.update_layout(
                        title='Most Active Hours',
                        yaxis= go.layout.YAxis(title="Total Messages"),
                        showlegend=False,
                        xaxis = go.layout.XAxis(title='Hours', tickangle=45, tickvals=list(range(24)))
                   )
    
    hourly_fig.update_traces(texttemplate='%{y:.2s}', textposition='outside')

    return  monthly_fig, daily_fig, weekly_fig, hourly_fig



def get_activity_map(df, user):
    """
    Plot activity map for each day and hour
    """
    if user.lower() != 'overall':
        df = df[df.id == user]

    df['period'] = df['hour'].astype(str) + '-' + ((df['hour'] + 1) % 24).astype(str)
    df = df.groupby(['DayName', 'period'])['message'].count().reset_index()
    df = df.fillna(1)
    # pivot = df.pivot_table(index='DayName', columns='period', values='message', aggfunc='count').fillna(0)
    fig = alt.Chart(df).mark_rect().encode(
            alt.X('period:O', axis=alt.Axis(title='hours')),
            alt.Y('DayName:O', axis=alt.Axis(title='days')),
            alt.Color('message:Q', scale=alt.Scale(scheme='goldorange'))
        )
    return fig

Overwriting utils/helpers.py


In [29]:
%%writefile utils/topic_model.py
from top2vec import Top2Vec
import re
from bs4 import BeautifulSoup
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
from wordcloud import WordCloud

nltk.download('wordnet')

global wnl
wnl = WordNetLemmatizer()

# list of custom stopwords
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't", 'hi', 'okay', 'ok', 'ohkay', 'bro', 'bye', 'thanks', 'thank', 'yeah', 'ya', \
            'u', 'ur', ])


# https://stackoverflow.com/a/47091490/4084039
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def preprocess_text(sentence:str):
    #a. remove html and url tags from text
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()

    #b.expand contracted terms
    sentence = decontracted(sentence)

    #c.remove non aplhabet characters
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)

    #d. lemmatize each word in sentence
    #e. and turn them into lower case
    #list of stop words: https://gist.github.com/sebleier/554280
    sentence = ' '.join(wnl.lemmatize(word.lower()) for word in sentence.
    split() if word.lower() not in stopwords)

    return sentence

def get_topics(df):
    """
    Preprocesses conversations to prepare it for Topic modelling
    Returns list of wordclouds of top two topics
    """

    df = df[df.message != '<Media omitted>']
    df = df[df.message != 'This message was deleted']

    documents = df['message'].apply(preprocess_text)
    model = Top2Vec(documents=documents.tolist(), speed="learn", workers=-1)

    num_topics = model.get_num_topics()
    if num_topics >= 2:
        topic_words, word_scores, topic_nums = model.get_topics(2)

    elif num_topics == 1:
        topic_words, word_scores, topic_nums = model.get_topics(1)
    
    else:
        return []

    clouds = []
    for topic in topic_words[:2]:
        wc = WordCloud(width=700, height=300, min_font_size=12, background_color='white')
        wc = wc.generate(' '.join(topic))
        clouds.append(wc)

    del model
    del documents
    return clouds

Overwriting utils/topic_model.py


In [31]:
%%writefile app.py
import os
import streamlit as st
from io import StringIO
from utils.preprocessing import preprocess
import altair as alt
from matplotlib import pyplot as plt
from numerize import numerize

from utils.helpers import (
    fetch_messages,
    fetch_stats, 
    fetch_active_users, 
    get_wordcloud, 
    most_common_emojis,
    get_timelines,
    timeline_stats,
    get_activity_map,
)

from utils.topic_model import get_topics

import seaborn as sns
sns.set()


PAGE_CONFIG = {"page_title":"App by Glad Nayak","page_icon":":smiley:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

def main():
    """
    Render UI on web app, fetch and display data using utils.py
    """
    st.title("WhatsApp Chat Analysis")
    uploaded_file = st.sidebar.file_uploader("Choose a file")
    if not uploaded_file:
            st.subheader('Upload exported text file to see analysis')
        
    if uploaded_file:
        try:
            # save the file, and pass the filename to preprocess function
            with open(os.path.join(".", uploaded_file.name),"wb") as f:
                f.write(uploaded_file.getbuffer())

            chats_with_date, chats = preprocess(uploaded_file.name)
        
        except:
            st.text('Failed to read the exported file. Try again')
            return -1

        # Get all users involved in conversation
        users = chats.id.unique().tolist()
        users.sort()
        users.insert(0, 'Overall')

        
        # 1. Show stats based on selected user
        user = st.sidebar.selectbox('Show analysis wrt', users)

        user_title = f'Showing Analysis for {user}' if user != 'Overall' else 'Showing Overall Analysis'
        st.subheader(user_title)

        headers = ['Members', 'Messages', 'Words', 'Media Uploaded', 'Links Shared']
        stats = [chats.id.nunique()]
        stats.extend(fetch_stats(chats, user))

        # don't show total members for personal conversations
        headers = headers[1:] if user != 'Overall' else headers
        stats = stats[1:] if user != 'Overall' else stats

        metrics = zip(headers, st.columns(len(headers)), stats)
        for header, column, stat in metrics:
            with column:
                st.metric(label=header, value=numerize.numerize(stat))


        # 2. display dataframe
        st.dataframe(fetch_messages(chats, user))


        # 3. plot activity map of user
        title = f'Activity map of {user}' if user != 'Overall' else 'Activity Map of Users'
        st.subheader(title)

        headers = ['Active Days', 'First Message on', 'Last Message on']
        for header, column, value in zip(headers, st.columns(3), timeline_stats(chats, user)):
            with column:
                st.metric(label=header, value=value)

        st.altair_chart(get_activity_map(chats, user), use_container_width=True)


        # 4. Show Overall stats
        if user == 'Overall':
            headers = ['Most Active', 'Most Active(%)']
            functions = [st.bar_chart, st.table]
            top_users, top_users_percent = fetch_active_users(chats)
        
            col1, col2 = st.columns(2)

            # plot most active users
            top_users = top_users[:10] 

            with col1:
                st.subheader(f'Top {len(top_users)} Active Users')
                bar_chart = alt.Chart(top_users).mark_bar(
                        cornerRadiusTopLeft=3,
                        cornerRadiusTopRight=3,
                ).encode(
                        x=alt.X('User:N', axis=alt.Axis(labelAngle=45)),
                        y='Messages:Q',
                        color=alt.Color('User:N', scale=alt.Scale(scheme='goldorange'), legend=None),
                )

                text = bar_chart.mark_text(
                            align='center',
                            dy=-5,
                            color='white'
                            ).encode(
                            text='Messages'
                        )
                            
                bar = (bar_chart + text).properties(height=400)
                st.altair_chart(bar, use_container_width=True)

            # plot percentage of active users
            with col2:
                st.subheader('Most Active Users(%)')
                st.dataframe(top_users_percent)


        # 5. Plot word clouds
        try:
            st.subheader('Word Cloud')
            wc = get_wordcloud(chats, user)
            fig, ax = plt.subplots()
            ax.imshow(wc)
            plt.axis('off')
            st.pyplot(fig)

        except:
            st.text("There's been lot of silence lately...")


        # 6. plot metrics on words
        header = 'Most Common Emojis'
        table = most_common_emojis(chats, user)
        st.subheader(header)
        try:
            st.altair_chart(table, use_container_width=True)

        except:
            import random
            filler = random.choice([
                            "A picture is worth a thousand words, but couldn't find either",
                            "Actions speak louder than words, hopefully!"
                ])
            st.text(filler)


        # 7. display timelines
        stats = get_timelines(chats_with_date, user)
        for timeline_plot in stats:
            st.plotly_chart(timeline_plot, use_container_width=True)

        
        # 8. display topics
        topics = get_topics(chats)
        if topics:
            st.subheader('Learning what members are talking about using Topic Modelling')
            for idx, wc in enumerate(topics):
                try:
                    st.subheader(f'Topic {idx+1}')
                    fig, ax = plt.subplots()
                    ax.imshow(wc)
                    plt.axis('off')
                    st.pyplot(fig)

                except:
                    st.text("")

if __name__ == '__main__':
	main()

Overwriting app.py


## Local Deployment using Local Tunnel - For testing

In [33]:
!streamlit run app.py --server.enableCORS=false &>/dev/null&

!lt --Bypass-Tunnel-Reminder --subdomain 'bornapp' --port 8501 &>/dev/null&

In [32]:
# kill app and clean up memory
st_id = !pgrep streamlit
!kill {st_id[0]}

lt_id = !pgrep lt
!kill {lt_id[0]}

/bin/bash: line 0: kill: {lt_id[0]}: arguments must be process or job IDs


## Deploying to Heroku

In [43]:
# first step is to make a requirements.txt file
#!pip freeze --local > requirements.txt
#!pip install pipreqs
!pipreqs --force .

INFO: Successfully saved requirements file in ./requirements.txt


setup.py and Procfile tell Heroku how to start our webapp
<br> source: https://gilberttanner.com/blog/deploying-your-streamlit-dashboard-with-heroku

In [7]:
%%writefile setup.py

mkdir -p ~/.streamlit/

echo "\
[general]\n\
email = \"your-email@domain.com\"\n\
" > ~/.streamlit/credentials.toml

echo "\
[server]\n\
headless = true\n\
enableCORS=false\n\
port = $PORT\n\
" > ~/.streamlit/config.toml

Writing setup.py


In [8]:
%%writefile Procfile

web: sh setup.sh && streamlit run app.py

Writing Procfile


In [ ]:
# for rest of the steps, we need to be logged in to heroku, either
# using heroku login, or heroku login -i.
!wget https://cli-assets.heroku.com/install.sh #heroku cmd line
!heroku login
!git add .
!git commit -m "deploy to heroku"
!git push heroku main
!heroku ps:scale web=1

In [45]:
# runtime.txt
!python --version

Python 3.7.12
